In [1]:
import pandas as pd

import transformers
from transformers import BertModel, BertTokenizer, RobertaConfig, RobertaModel
import torch
import torch.nn as nn
from dont_patronize_me import DontPatronizeMe
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader

RANDOM_SEED = 42

In [ ]:
dpm = DontPatronizeMe('.', '.')
dpm = DontPatronizeMe('.', 'dontpatronizeme_pcl.tsv')

dpm.load_task1()

dpm2 = DontPatronizeMe('.', '.')
dpm2 = DontPatronizeMe('.', 'dontpatronizeme_categories.tsv')

dpm2.load_task2()

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [ ]:
df = dpm.train_task1_df
df_cate = dpm2.train_task2_df

In [ ]:
df

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [ ]:
df['len_text'] = df.apply(lambda x: len(x['text']), axis=1)

In [ ]:
df

NameError: name 'df' is not defined

In [ ]:

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

### Sample text to visualise tokenisation

In [ ]:
sample_txt = 'Hello! I love that you are so poor.'
tokens_sample = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens_sample)
print(token_ids) 
print(len(token_ids)) 
print(tokens_sample)

[8667, 106, 146, 1567, 1115, 1128, 1132, 1177, 2869, 119]
10
['Hello', '!', 'I', 'love', 'that', 'you', 'are', 'so', 'poor', '.']


In [ ]:
print(tokenizer.sep_token, tokenizer.sep_token_id) # end of sentence marker
print(tokenizer.cls_token, tokenizer.cls_token_id) # start of sentence token 
print(tokenizer.pad_token, tokenizer.pad_token_id) # token for padding
print(tokenizer.unk_token, tokenizer.unk_token_id) # token for unknown

[SEP] 102
[CLS] 101
[PAD] 0
[UNK] 100


Then, we do embedding on the tokens. 

In [ ]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  truncation = True, # truncate examples to max length 
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  padding = "max_length", 
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

print(encoding.keys()) # dict_keys(['input_ids', 'attention_mask'])
print(encoding.input_ids) 

dict_keys(['input_ids', 'attention_mask'])
tensor([[ 101, 8667,  106,  146, 1567, 1115, 1128, 1132, 1177, 2869,  119,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])


In [ ]:
print(encoding['attention_mask'])

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
class SentimentClassifier(nn.Module):
  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
df_train, df_test = train_test_split(
  df,
  test_size=0.3, 
  random_state = RANDOM_SEED
)

df_test, df_val = train_test_split(
  df_test,
  test_size=0.5, 
  random_state = RANDOM_SEED
)

df_train.shape, df_val.shape, df_test.shape

((7328, 7), (1571, 7), (1570, 7))

### Dataloader

In [ ]:
class MyDataset(Dataset):

  def __init__(self,dataframe, tokenizer, max_len):
    self.x=dataframe.loc[:,['text']]
    self.y=dataframe.loc[:,['label']]

    self.x = np.array(self.x)
    self.y = np.array(self.y)
    
    # self.x_train=torch.from_numpy(self.x)
    # self.y_train=torch.from_numpy(self.y)
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.y)
  
  def __getitem__(self,idx):
    input = str(self.x[idx])
    labels = self.y[idx]

    encoding = tokenizer.encode_plus(
      input,
      max_length= self.max_len,
      truncation = True, # truncate examples to max length 
      add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=False,
      padding = "max_length", 
      return_attention_mask=True,
      return_tensors='pt')  # Return PyTorch tensor

    return {
      'text': input,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(labels, dtype=torch.long)
    }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = MyDataset(
    dataframe = df,
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [ ]:
BATCH_SIZE = 16
MAX_LEN = 160 # can change this 
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)

# Classification

In [ ]:
from transformers import RobertaModel, RobertaConfig

configuration = RobertaConfig()
model = RobertaModel(configuration)
output = model(
  input_ids=encoding['input_ids'],
  attention_mask=encoding['attention_mask']
)

print (output[1].shape)

torch.Size([1, 768])


In [ ]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

else:
    device = "cpu"

print(device)

False
cpu


In [ ]:
class SentimentClassifier(nn.Module):
  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    # self.bert = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    self.model = model
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(768, n_classes)
    
  def forward(self, input_ids, attention_mask):
    output = self.model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    
    print(output[1])
    print(output[1].shape)

    output = self.drop(output[1])
    return self.out(output)

## Training


In [ ]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
EPOCHS = 1
device = 'cpu'

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_data_loader) * EPOCHS

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    SentimentClassifier(2),
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler = None,
    n_examples = len(df_train)
  )

print(f'Train loss {train_loss} accuracy {train_acc}')
